# Application of  Automatic Differentiation in Finance Using AlgoPy (PyData DC 2016)
#####  Li Guo, Nicholas Liu, Nadia Udler

### Introduction of Automatic Differentiation Using Taylor Expansion

Automatic differentiation, or algorithmic differentiation (AD), is a set of techniques to evaluate the derivative of a function specified by a computer program. Fundamental to AD is the application of the chain rule for computing derivatives.

<br>
`AlgoPy` is a Python module for automatic differentiation. This tutorial shows how to calculate one-dimensional Taylor series expansion and partial derivatives of function by `AlgoPy`.

<br>
Recall Taylor expansion of an infinitely differentiable function to approximate function value $f(x)$ about a point a:

<br>
$$f(x)=f(a)+\frac{f'(a)}{1!}(x-a)+\frac{f''(a)}{2!}(x-a)^2+\cdots=\sum^{\infty}_{n=0}\frac{f^{(n)}(a)}{n!}(x-a)^n$$

<br>
For example, consider this function:
$$f(x)=x^3+\sin(2x)-e^{-x}$$

<br>
In this case we can compute derivatives analytically:

<br>
$$\begin{align*} 
f'(x)&=3x^2+2\cos(2x)+e^{-x} \\
f''(x)&=6x-4\sin(2x)-e^{-x} \\
f^{(3)}(x)&=6x-8\cos(2x)+e^{-x}
\end{align*}$$

<br>
For the above function, we can calculate the first 4 Taylor series expansion with $a=3$ and $x=3.5$ as below:

<br>
$$\begin{align*} 
f(a)&=26.671 \\
\frac{f'(x)}{1!}(x-a)&=14.485 \\
\frac{f''(a)}{2!}(x-a)^2&=2.383 \\
\frac{f^{(3)}(a)}{3!}(x-a)^3&=-0.034
\end{align*}$$

<br>
With these derivatives we can easily approximate function value at any given x.

<br>
However, when target function is too complicated to be differentiated manually, we can use AD to get the coefficients. The code and result (same as in manually calculated example):

In [1]:
import numpy as np
from algopy import UTPM

# Target function
def f(x):
    return (np.power(x, 3) + np.sin(2 * x) - np.exp(-x))

D = 10                     # Number of coefficient parts (expansion parts)
P = 1                      # Used to evaluate several Taylor at once, no effects here
x = UTPM(np.zeros((D, P))) # UTPM stands for Univariate Taylor Polynomial of Matrices
x.data[0, 0] = 3           # a = 3
x.data[1, 0] = 0.5         # x - a = 0.5
y=f(x)
print(y.data[:, 0])

[  2.66707974e+01   1.44850638e+01   2.38348437e+00  -3.39911505e-02
  -1.17719662e-02   8.01438444e-03   3.86996632e-04  -1.90432803e-04
  -6.93477130e-06   2.64623988e-06]


### Computing Prices and Sensitivities of European Option

Next we look at computing gradient vector, Hessian matrix and Jacobian matrix with `AlgoPy`. These are used in quantitative finance field frequently. 

<br>
Suppose the target function is multidimensional ${\rm I\!R}^n\rightarrow{\rm I\!R}$. We would like to compute partial derivatives of first and second order. In many cases it is difficult to compute analytical derivatives of the function (the analytical form of the function is not available , or too complicated). Then  AD  can be used to compute gradient vector, Jacobian and Hessian matrices of the function.

<br>
As an example we look at  Black-Scholes (BS) Formula for pricing European call option on stock. This  contracts give the owner the right, but not the obligation, to buy the underlying security at a specific price, known as the strike price ($K$), on the option's expiration date (time $T$). The below expression gives the price ($C$) of the contract, where $\sigma$ is volatility of the underlying, $S$ is the current price of the underlying, and $r$ is a risk free rate.

<br>
$$\begin{align*} 
C(S, \sigma, T, r)&=SN(d_1)+Ke^{-rT}N(d_2) \\
d_1&=\dfrac{\ln(\dfrac{S}{K})+(r+\dfrac{\sigma^2}{2})T}{\sigma\sqrt{T}} \\
d_2&=d_1-\sigma\sqrt{T}
\end{align*}$$

<br>
First-order sensitivities:

<br>
$$delta=\frac{\partial C}{\partial S} \qquad vega=\frac{\partial C}{\partial \sigma} \qquad theta=\frac{\partial C}{\partial T} \qquad rho=\frac{\partial C}{\partial r}$$

<br>
Second-order sensitivities:

<br>
$$\begin{align*}
gamma=\frac{\partial^2 C}{\partial S^2} \qquad vanna&=\frac{\partial^2 C}{\partial S\partial \sigma} \qquad vomma=\frac{\partial^2 C}{\partial \sigma^2} \\
\\
charm=-\frac{\partial^2 C}{\partial T \partial S} \qquad veta&=\frac{\partial^2 C}{\partial \sigma \partial T} \qquad vera=\frac{\partial^2 C}{\partial \sigma \partial r}
\end{align*}$$


<br>
Then we calculate gradient vector and Hessian matrix of the BS function with `AlgoPy`.The sensitivities computed above will be the elements of the gradient and Hessian. For this purpose we define BS function as a Python function and call the `AlgoPy` to find  gradient and Hessian. 
One obstacle we encounter when applying AD to BS function in Python: BS function would be calling built-in function (`.norm.cdf(x)` from `scipy`), that computes cumulative normal distribution `N(x)`. However, AD cannot be applied to built-in functions (chain rule cannot be applied to the function if it cannot be accessed) . We use our version of normal cds function (algorithms is available from many sources, for example see Numerical methods in C by Press, Teukolsky et al).

In [2]:
from numpy import *
from algopy import UTPM

def erf(x):
    """Complementary error function"""
    z = abs(x)
    t = 1 / (1 + 0.5 * z)
    r = t * exp(-z*z-1.26551223+t*(1.00002368+t*(.37409196+
        t*(.09678418+t*(-.18628806+t*(.27886807+
        t*(-1.13520398+t*(1.48851587+t*(-.82215223+
        t*.17087277)))))))))
    if (x >= 0):
        return r
    else:
        return 2 - r
    
def N(x):
    """Normal cumulative density function"""
    return 1 - 0.5 * erf(x / (2 ** 0.5))

Function C computes a price of European call option, where the input x is a vector of $[S,\sigma,T,r]$ and strike price equals 10.

In [3]:
def C(x):
    K, S, sigma, T, r = 10, x[0], x[1], x[2], x[3]
    d1 = (log(S / K) + (r + power(sigma, 2) / 2)) / (sigma * sqrt(T))
    d2 = d1 - x[1] * sqrt(x[2])
    return S * N(d1) - K * exp(-r * T) * N(d2)

### Jacobian Matrix

The Jacobian matrix (also the gradient) in our case is

<br>
$$\begin{bmatrix}
\dfrac{\partial C}{\partial S} &\dfrac{\partial C}{\partial \sigma} &\dfrac{\partial C}{\partial T} &\dfrac{\partial C}{\partial r}
\end{bmatrix}
$$

<br>
The code to calculate Jacobian matrix is as below:

In [4]:
x = UTPM.init_jacobian([12, 0.2, 1, 0.03]) # S=12, σ=0.2, T=1, r=0.03
y = C(x)
J = UTPM.extract_jacobian(y)
print('First-order Greeks:\n', J)

First-order Greeks:
 [ 0.87730282  2.43829855  0.48601709  8.07291282]


### Hessian Matrix

The Hessian matrix in our case is

<br>
$$\begin{bmatrix}
    \dfrac{\partial^2 C}{\partial S^2} &\dfrac{\partial^2 C}{\partial \sigma \partial S} &\dfrac{\partial^2 C}{\partial T \partial S} &\dfrac{\partial^2 C}{\partial r \partial S} \\
    \dfrac{\partial^2 C}{\partial S \partial \sigma}&\dfrac{\partial^2 C}{\partial \sigma^2} &\dfrac{\partial^2 C}{\partial T \partial \sigma} &\dfrac{\partial^2 C}{\partial r \partial \sigma} \\
    \dfrac{\partial^2 C}{\partial S \partial T} &\dfrac{\partial^2 C}{\partial \sigma \partial T} &\dfrac{\partial^2 C}{\partial T^2} &\dfrac{\partial^2 C}{\partial r \partial T} \\
    \dfrac{\partial^2 C}{\partial S \partial r} &\dfrac{\partial^2 C}{\partial \sigma \partial r} &\dfrac{\partial^2 C}{\partial T \partial r} &\dfrac{\partial^2 C}{\partial r^2}
\end{bmatrix}$$


<br>
The code to calculate Hessian matrix is as below:

In [5]:
x = UTPM.init_hessian([10, 0.2, 1, 0.03]) # S=10, σ=0.2, T=1, r=0.03
y = C(x)
H = UTPM.extract_hessian(4, y) # H is of size 4 by 4
print('Hessian matrix and second-order Greeks:\n', H)

Hessian matrix and second-order Greeks:
 [[  0.19340522  -0.09682358   0.04830366   1.93405216]
 [ -0.09682358   0.24204836   1.81257802  -4.8349186 ]
 [  0.04830366   1.81257802  -0.25537928   4.99071985]
 [  1.93405216  -4.8349186    4.99071985  14.29480367]]


C:\Anaconda3\lib\site-packages\algopy\utpm\utpm.py:1798: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  S = numpy.zeros((N,M), dtype=x.dtype)


### Comparing with Closed-Form Solution

One way of checking these results is to use the theoretical derivation of Greeks from Black-Scholes-Merton model. See closed-form formula of Greeks [here](https://en.wikipedia.org/wiki/Black–Scholes_model#The_Greeks).

#### Checking First-Order Greeks

Consider computing the derivative of option price with respect to the underlying asset's price, i.e., $delta$: its closed form is just the normal cumulative distribution function $N(d_1)$, where $d_1=\dfrac{\ln(\dfrac{S}{K})+(r+\dfrac{\sigma^2}{2})T}{\sigma\sqrt{T}}$. We can use built-in normal cumulative density function  from `SciPy` to calculate closed-form $delta$ as below:

In [6]:
from scipy.stats import norm
K, S, sigma, T, r = 10, 12, 0.2, 1, 0.03
d1 = (log(S / K) + (r + power(sigma, 2) / 2)) / (sigma * sqrt(T))
print("Closed-form delta using scipy.stats.norm is:\n", norm.cdf(d1))

Closed-form delta using scipy.stats.norm is:
 0.877302590601


Also, we can use our own normal cumulative density function `N(x)` to check our result. The result is as below:

In [7]:
print("Closed-form delta using N(x) is:\n", N(d1))

Closed-form delta using N(x) is:
 0.877302582172


It turns out that the above two $delta$ values are almost the same as what we get from automatic differentiation, see below:

In [8]:
print("Delta calculated using AD is:\n", J[0])

Delta calculated using AD is:
 0.877302823318


#### Checking Second-Order Greeks

We take $charm$ of call option as an example. It is a rate of change of $delta$ over time. Its closed form is $-\phi(d_1)\dfrac{2rT-d_2\sigma\sqrt{T}}{2T\sigma\sqrt{T}}$, where $\phi(x)$ denotes the standard normal probability density function, i.e., $\dfrac{1}{2\sqrt{\pi}}e^{-\frac{x^2}{2}}$, and $d_2 = d_1-\sigma\sqrt{T}$.

<br>
We use built-in normal probability density function from `SciPy` to calculate closed-form $charm$ as follows:

In [9]:
K, S, sigma, T, r = 10, 10, 0.2, 1, 0.03
d1 = (log(S / K) + (r + power(sigma, 2) / 2)) / (sigma * sqrt(T))
d2 = d1 - sigma * sqrt(T)
charm = -norm.pdf(d1) * (2 * r * T - d2 * sigma * sqrt(T)) / (2 * T * sigma * sqrt(T))
print("Closed-form charm using scipy.stats.norm is:\n", charm)

Closed-form charm using scipy.stats.norm is:
 -0.0483335146004


Again, the closed-form result is pretty close to the result obtained from AD,

In [10]:
# Be cautious of the minus sign in the definition of charm
print("Charm calculated using AD is:\n", -H[0][2]) 

Charm calculated using AD is:
 -0.0483036618516


This concludes our tutorial. For more information and a general tutorial of `AlgoPy`, check its official website at https://pythonhosted.org/algopy/